[Reference](https://medium.com/mlearning-ai/object-oriented-programming-and-ml-model-development-in-python-ada4bf76529b)

In [1]:
class HumanBeings():
  """ 
  The base class for all the human beings.
  """

  def __init__(self, name, height, gender, contact = None):
     self.name = name
     self.height = height
     self.gender = gender
     self._contact = contact # A private attribute, please dont alter it from outside
 
    

  def say_the_word(self, word):
    """
    A method for performing the action of speaking the passed word.
    Parameters
    ----------
    word : str
    Insert the word to be spoken.
    Returns
    --------
    A string  
    """
    
    
    return f"{self.name} says {word}"

# Some OOPs concepts
- Inheritance
- Method Overloading
- Abstract Classes and methods



In [2]:
from abc import ABC,abstractmethod
import inspect


"""Base class for all the predictors.
   Since we cant create an instance of abstract class, we assume parameters (attributes) are initialzied by the child model class. """

class BasePredictor(ABC):
   
    @abstractmethod
    def fit(self, X, y):
        """Fit predictor.
        Parameters
        ----------
        X : {ndarray, sparse matrix} of shape (n_samples, n_features)
            The input samples.
        y : ndarray of shape (n_samples,)
            The input target value. 
        Returns
        -------
        self : object
            Fitted estimator.
        """
        pass

    @abstractmethod
    def predict(self, X):
        """
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            
        Returns
        -------
        y_pred : ndarray of shape (n_samples,)
        """
        pass

    
    # copied from sklearn 
    @classmethod
    def _get_param_names(cls):
        """Get parameter names for the estimator"""
        # fetch the constructor or the original constructor before
        # deprecation wrapping if any
        init = getattr(cls.__init__, "deprecated_original", cls.__init__)
        if init is object.__init__:
            # No explicit constructor to introspect
            return []

        # introspect the constructor arguments to find the model parameters
        # to represent
        init_signature = inspect.signature(init)
        # Consider the constructor parameters excluding 'self'
        parameters = [
            p
            for p in init_signature.parameters.values()
            if p.name != "self" and p.kind != p.VAR_KEYWORD
        ]
        for p in parameters:
            if p.kind == p.VAR_POSITIONAL:
                raise RuntimeError(
                    "scikit-learn estimators should always "
                    "specify their parameters in the signature"
                    " of their __init__ (no varargs)."
                    " %s with constructor %s doesn't "
                    " follow this convention." % (cls, init_signature)
                )
        # Extract and sort argument names excluding 'self'
        return sorted([p.name for p in parameters])

    #copied from sklearn
    def get_params(self, deep=True):
        """
        Get parameters for this estimator.
        Parameters
        ----------
        deep : bool, default=True
            If True, will return the parameters for this estimator and
            contained subobjects that are estimators.
        Returns
        -------
        params : dict
            Parameter names mapped to their values.
        """
        out = dict()
        for key in self._get_param_names():
            value = getattr(self, key)
            if deep and hasattr(value, "get_params"):
                deep_items = value.get_params().items()
                out.update((key + "__" + k, val) for k, val in deep_items)
            out[key] = value
        return out

    
    def reset(self):
       """A method for reseting the predictor"""   
       new = self.__class__(**self.get_params())
       return new

    
    def load_params(self, params):
      """A method to load model configurations.
      
      Parameters
      -----------
      params : dict of parameters
  
      Returns
      ---------
      A new model instance with the new parameters.  
      """

      self = self.__class__(**params)
      print("params loaded")
      
      return self

In [3]:
import numpy as np
from sklearn.utils.validation import check_X_y, check_array

class LinearRegression(BasePredictor):
      def __init__(self, set_intercept = True):
        self.set_intercept = set_intercept
        self.intercept = 0
          
      
      def fit(self,X,y):
         X, y = check_X_y(X, y) #Checking if both X & y has correct shape and converting X, y into 2d and 1d array (even pandas dataframe gets converted into arrays)
         if self.set_intercept == True:
            X_ = np.c_[np.ones((X.shape[0],1)), X] #adding x0 = 1
         else:
           X_ = X   
         self.beta = np.linalg.inv(X_.T.dot(X_)).dot(X_.T).dot(y)
         if self.set_intercept == True:
          self.coefficients = self.beta[1:]
          self.intercept = self.beta[0]
         else:
            self.coefficients = self.beta
         return self 

      def predict(self,X):
        X_ = check_array(X) # Validate the input, convert into 2d numpy array
        return X_@self.coefficients + self.intercept

In [4]:
from tensorflow import keras
from keras.models import Sequential
from keras.layers import BatchNormalization, Dense, Dropout
from sklearn.utils import check_array

class FFNNPredictor(BasePredictor):

   """
   Parameters
   -----------
   hidden_layers : an array of integers  (Default = [])
   Initialize the hidden layers of the neural network by passing a list of neurons per hidden layer.
   example: 
   If hidden_layers = [10, 10 ,2], then the neural network will have 1st hidden layer with 10 neurons, second with 10 neurons and the third hidden layer with 2 neurons.
   
   activation: an array of integers  (Default = [])
   Set the type of activation function for all the neurons present in each of the hidden layers.
   example: activation = ["relu", "relu", "relu"] will set all the three layers to have relu activation function.
   Note: The  size of the activation array should be same as the hidden_layers.
   
   dropout: float (between 0 and 1, Default = 0)
   randomly sets input units to 0 with a frequency of dropout at each step during training time, which helps prevent overfitting.
   The dropout layers are present in between all the subsequent layers if the model and has the same dropout rate given by dropout.
 
   training parameters:
   
  
   epochs: Integer (Default = 1)
   Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided. 
   Note that in conjunction with initial_epoch, epochs is to be understood as "final epoch". 
   The model is not trained for a number of iterations given by epochs, but merely until the epoch of index epochs is reached.
   batch_size: Integer or None. 
   Number of samples per gradient update. If unspecified, batch_size will default to 32.
   The batch size is a hyperparameter that defines the number of samples to work through before updating the internal model parameters. 
   Note: The final layer has only one neuron with identity activation. (For regression)
 
   """
   def __init__(self, hidden_layers = [], activation = [], dropout = 0, epochs = 1, batch_size = None):
        
        self.activation = activation
        self.hidden_layers = hidden_layers
        self.epochs = epochs
        self.batch_size = batch_size
        self.model = Sequential()
        self.dropout = dropout
        for layer in range(len(hidden_layers)):
            #sequentially add layers to the model
            self.model.add(Dense(self.hidden_layers[layer], self.activation[layer], kernel_initializer = keras.initializers.glorot_uniform()))
            self.model.add(BatchNormalization())
            self.model.add(Dropout(self.dropout))
        
        #final regression layer
        self.model.add(Dense(1))
        # setting up the type of gradient descent
        self.model.compile(loss = "mse" , optimizer="adam")
       
        
   def fit(self, X, y, **kwargs):
           
            """
            Fit FFNN model. 
        
            Parameters
            ----------
            X : {array-like, sparse matrix} of shape (n_samples, n_features)
             Training data. The column "timestamp" will be removed if it is found. (When X is a pandas dataframe) 
            
            y : array-like of shape (n_samples,) or (n_samples, n_targets)
            Target values."""

            try: #if X is a pandas object with timestamp column
                if "timestamp" in X.columns:
                    X = X.drop("timestamp", axis = 1)
            except:
                pass
            X = check_array(X)
            y = check_array(y, ensure_2d = False)         
            return self.model.fit(X, y, epochs = self.epochs, batch_size = self.batch_size, **kwargs) 

   def predict(self, X):
        
            """
            Parameters
            ----------
            X : array-like or sparse matrix, shape (n_samples, n_features)
            
            Returns
            --------
            An array of model estimates for input X.
            """



            try: #if X is a pandas object with timestamp column
                if "timestamp" in X.columns:
                    X = X.drop("timestamp",axis = 1)
            except:
                pass
            #converting into numpy array
            X = check_array(X) 

            return  self.model.predict(X)    
   
   def summary(self):
       """Once a model is "built", you can call its summary() method to display its contents"""

       return self.model.summary()

In [5]:
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_boston

X, y = load_boston(return_X_y=True)
print(X.shape)

(506, 13)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

In [6]:
lr = LinearRegression()
lr.get_params()

{'set_intercept': True}

In [7]:
lr.fit(X, y)

In [8]:
pred = lr.predict(X)
mean_squared_error(pred, y)

21.894831181729206

In [9]:
from sklearn.linear_model import LinearRegression as LR
lr_2 = LR()
lr_2.fit(X,y)
pred2 = lr_2.predict(X)
mean_squared_error(pred2,y)

21.894831181729202